# Evaluate the predicted targets of Rck2 and Cdc28 - June 22, 2018

In [1]:
import os
import pandas as pd
import numpy as np

data_dir = os.path.join('..', '..', 'data')
eval_dir = os.path.join(data_dir, 'evaluation')
tps_dir = os.path.join('..', '..', 'Results', 'network_visualization', 'kanshin_072117')
annot_dir = os.path.join('..', 'Cytoscape_Visualization', 'kanshin_072117')

rck2_file = os.path.join(eval_dir, 'TableS2_Romanov.xlsx')
id_file = os.path.join(data_dir, 'SGD_id_map.txt')
rck2_tps_dir_file = os.path.join(tps_dir, 'rck2-directed-neighbors.txt')
rck2_tps_act_file = os.path.join(tps_dir, 'rck2-activated-neighbors.txt')
annot_file = os.path.join(annot_dir, 'kanshin2015-cytoscape-annotations.txt')

## Load and format the Rck2 target data

Only load the following columns:
- **0**: Protein name
- **1**: PhosphoSite
- **16**: SETUP SR wt +/- 5min 0.5M NaCl Average L/H-ratio
- **71**: SETUP rck2 wt vs. rck2Δ + 5min 0.5M NaCl Average L/H-ratio

Have to skip the initial rows because the Δ doesn't parse correctly.

Romanov Table S6 defines the strains:
- **WR209**: W303, Mat a; lys1::kanMX arg4::kanMX CAN1 leu2-3,112 trp1-1 ura3-1 ade2-1 his3-11,15
- **MJ243**: W303, Mat a; rck2::HIS3 lys1::kanMX arg4::kanMX CAN1 leu2-3,112 trp1-1 ura3-1 ade2-1 his3-11,23

Romanov Table S8 defines the ratios:
- **SETUP SR wt +/- 5min 0.5M NaCl Average L/H-ratio**: 0.5M NaCl for 5mins / no stress
- **SETUP rck2 wt vs. rck2Δ + 5min 0.5M NaCl Average L/H-ratio**: MJ243 +5mins 0.5M NaCl / WR209 +5mins 0.5M NaCl

In [2]:
headers = ['Protein', 'Phosphosite', 'WT 5min L/H ratio', 'rck2 del 5min L/H ratio']
rck2_df = pd.read_excel(rck2_file, header=None, skiprows=[0,1,2], parse_cols=[0,1,16,71], names=headers)

# check the length and number of non-missing data points
assert len(rck2_df) == 16413
assert sum(np.isnan(rck2_df['WT 5min L/H ratio'])) == 8356
assert sum(np.isnan(rck2_df['rck2 del 5min L/H ratio'])) == 11579

In [3]:
# remove rows missing values in either condition
filt_rck2_df = rck2_df.dropna()
assert len(filt_rck2_df) == 2794

# spot check some proteins' values
assert filt_rck2_df[(filt_rck2_df['Protein'] == 'ABP140') & (filt_rck2_df['Phosphosite'] == '326(S)')]['WT 5min L/H ratio'].values[0] == 1.21
assert filt_rck2_df[(filt_rck2_df['Protein'] == 'ABP140') & (filt_rck2_df['Phosphosite'] == '326(S)')]['rck2 del 5min L/H ratio'].values[0] == 1.00

assert filt_rck2_df[(filt_rck2_df['Protein'] == 'ZUO1') & (filt_rck2_df['Phosphosite'] == '50(S)')]['WT 5min L/H ratio'].values[0] == 0.87
assert filt_rck2_df[(filt_rck2_df['Protein'] == 'ZUO1') & (filt_rck2_df['Phosphosite'] == '50(S)')]['rck2 del 5min L/H ratio'].values[0] == 1.01

## Filter the Rck2 target data
- Must have >= 2 fold change in wild type when stimulated with NaCl
- Must have Rck2 defect >= 1.5 fold, that is, (rck2 del)/(wild type) <= 1/1.5

Phosphosites that are not observed in the Romanov wild type setup will be processed separately.

In [4]:
filt_rck2_df = filt_rck2_df[filt_rck2_df['WT 5min L/H ratio'] >= 2]
assert len(filt_rck2_df) == 408

# Use a threshold of 2/3 intead of 0.5
filt_rck2_df = filt_rck2_df[filt_rck2_df['rck2 del 5min L/H ratio'] <= 1/1.5]
assert len(filt_rck2_df) == 140

## Format Rck2 target phosphosites
- Split di- and tri-phosphorylated peptides to match individual sites
- Map gene symbols to ORF identifiers

Track the sites, but the predictions will be matched at the protein level.

In [5]:
# Format the Romanov phosphosite to match the Kanshin phosphosite, including splitting sites
def format_site(site):
    site = site.rstrip(')')
    tokens = site.split('(')
    return tokens[1] + tokens[0]

def format_sites(sites):
    return map(format_site, sites.split(','))

In [6]:
# Test the site formatting
print(format_sites('312(S)'))
print(format_sites('233(T),236(S)'))

['S312']
['T233', 'S236']


In [7]:
id_df = pd.read_csv(id_file, sep='\t', header=None, names=['ORF', 'Symbol'])
assert len(id_df) == 8061
assert len(id_df['ORF'].unique()) == 8061
assert len(id_df['Symbol'].unique()) == 8061
id_map = dict(zip(id_df['Symbol'], id_df['ORF']))
# Make sure ORF ids can always map back to themselves
orf_orf_map = dict(zip(id_df['ORF'], id_df['ORF']))
id_map.update(orf_orf_map)

# Manually add SDH8 / YBR269C https://www.yeastgenome.org/locus/S000000473
# Manually add TRS65 / YGR166W https://www.yeastgenome.org/locus/S000003398
# Romanov table uses synonyms
assert 'FMP21' not in id_map
id_map['FMP21'] = 'YBR269C'
assert 'KRE11' not in id_map
id_map['KRE11'] = 'YGR166W'

rev_id_map = dict(zip(id_df['ORF'], id_df['Symbol']))

print('{} rck2 defective phosphosites before splitting di- and tri-phosphorylated'.format(len(filt_rck2_df)))

rck2_def_sites = set()
rck2_def_prots = set()

for index, row in filt_rck2_df.iterrows():
    for site in format_sites(row['Phosphosite']):
        orf = id_map[str(row['Protein'])]
        rck2_def_prots.add(orf)
        rck2_def_sites.add(orf + '::' + site)

print('{} rck2 defective phosphosites after splitting di- and tri-phosphorylated'.format(len(rck2_def_sites)))
print('{} proteins (ORFs) with a rck2 defective phosphosite'.format(len(rck2_def_prots)))

# confirm the proteins we discuss in the manuscript, expect the special case PIK1, appear in this list
# FPK1 / YNR047W
assert 'YNR047W' in rck2_def_prots
# ROD1 / YOR018W
assert 'YNR047W' in rck2_def_prots
# YLR257W
assert 'YNR047W' in rck2_def_prots

140 rck2 defective phosphosites before splitting di- and tri-phosphorylated
146 rck2 defective phosphosites after splitting di- and tri-phosphorylated
113 proteins (ORFs) with a rck2 defective phosphosite


## Supplement Rck2 target phosphosites

We also manually noted that PIK1 396(S) has a phospho defect in rck2 deletion.  It significantly increases in phosphorylation in the Kanshin study, is not observed in the wild type condition in the Romanov study, and has (rck2 del)/(wild type)=0.29 in the Romanov study.  Here, we systematically add all proteins that have significant changes on at least one phoshosite in the Kanshin data, are not observed in the Romanov wild type setup, and have rck2 defects.  We do not require the significant change in the Kanshin dataset to be an increase in phosphorlyation because that information is not easily accessible.

Also derive the proteins that have an rck2 defect and are observed in the Kanshin dataset.  We will use these proteins to calculate recall.

In [8]:
# Load the proteins with significantly changing phosphosites from Kanshin
kanshin_df = pd.read_csv(annot_file, sep='\t')

all_kanshin_prots = set(kanshin_df.loc[kanshin_df['SigPeptide1'].notnull() | kanshin_df['InsigPeptide1'].notnull(), 'Protein'].values)
assert len(all_kanshin_prots) == 1596

sig_kanshin_df = kanshin_df[kanshin_df['SigPeptide1'].notnull()]
assert len(sig_kanshin_df) == 784
assert set(sig_kanshin_df['NodeType'].values) == {'Excluded', 'SigPrize'}

sig_kanshin_prots = set(sig_kanshin_df['Protein'].values)
assert len(sig_kanshin_prots) == 784

In [9]:
# Confirm the Rck2 dataset has not yet been filtered
assert len(rck2_df) == 16413
assert sum(np.isnan(rck2_df['WT 5min L/H ratio'])) == 8356
assert sum(np.isnan(rck2_df['rck2 del 5min L/H ratio'])) == 11579

# Select phosphosites for which the wild type response is not observed and there is a phosphorylation defect in rck2 deletion
filt_rck2_df = rck2_df[rck2_df['WT 5min L/H ratio'].isnull()]
assert len(filt_rck2_df) == 8356

filt_rck2_df = filt_rck2_df[filt_rck2_df['rck2 del 5min L/H ratio'] <= 1/1.5]
assert len(filt_rck2_df) == 275

assert filt_rck2_df['Protein'].values[0] == 'ABP140'
assert filt_rck2_df['Phosphosite'].values[0] == '321(S)'

In [10]:
print('{} proteins (ORFs) with a rck2 defective phosphosite before adding significant Kanshin proteins'.format(len(rck2_def_prots)))

for index, row in filt_rck2_df.iterrows():
    orf = id_map[str(row['Protein'])]
    if orf in sig_kanshin_prots:
        rck2_def_prots.add(orf)

print('{} proteins (ORFs) with a rck2 defective phosphosite after adding significant Kanshin proteins'.format(len(rck2_def_prots)))
# PIK1 / YNL267W
assert 'YNL267W' in rck2_def_prots

# Proteins with rck2 phosphorylation defect by either definition and significant or insignificant response in Kanshin
kanshin_rck2_def_prots = rck2_def_prots.intersection(all_kanshin_prots)
print('{} proteins (ORFs) with a rck2 defective phosphosite and observed in Kanshin'.format(len(kanshin_rck2_def_prots)))

113 proteins (ORFs) with a rck2 defective phosphosite before adding significant Kanshin proteins
204 proteins (ORFs) with a rck2 defective phosphosite after adding significant Kanshin proteins
193 proteins (ORFs) with a rck2 defective phosphosite and observed in Kanshin


## Compare TPS Rck2 targets and Rck2 phosphorylation defects

Consider the directed targets of Rck2 seprately from the directed, activated targets.  These ratios could be interpreted and the precision and recall for the TPS Rck2 target predictions.  However, TPS predicts direct Rck2 targets and Rck2 defects can be indirect.  This is in part why the recall is low.

In [11]:
rck2_tps_dir_df = pd.read_csv(rck2_tps_dir_file, sep='\t', header=None, names=['ORF','Symbol'])
rck2_tps_dir_targs = set(rck2_tps_dir_df['ORF'].values)
assert len(rck2_tps_dir_targs) == 13

rck2_tps_dir_targs_def = rck2_def_prots.intersection(rck2_tps_dir_targs)
print('rck2 defect and TPS Rck2 directed target: {}'.format(len(rck2_tps_dir_targs_def)))
print('{} of rck2 defect'.format(float(len(rck2_tps_dir_targs_def))/len(rck2_def_prots)))
print('{} of TPS Rck2 directed target'.format(float(len(rck2_tps_dir_targs_def))/len(rck2_tps_dir_targs)))
for orf in sorted(rck2_tps_dir_targs_def):
    print('{}\t{}').format(orf, rev_id_map[orf])

rck2_tps_dir_targs_def_kanshin = kanshin_rck2_def_prots.intersection(rck2_tps_dir_targs)
print('\nrck2 defect Kanshin and TPS Rck2 directed target: {}'.format(len(rck2_tps_dir_targs_def_kanshin)))
print('{} of rck2 defect'.format(float(len(rck2_tps_dir_targs_def_kanshin))/len(kanshin_rck2_def_prots)))
print('{} of TPS Rck2 directed target'.format(float(len(rck2_tps_dir_targs_def_kanshin))/len(rck2_tps_dir_targs)))
for orf in sorted(rck2_tps_dir_targs_def_kanshin):
    print('{}\t{}').format(orf, rev_id_map[orf])

rck2_tps_act_df = pd.read_csv(rck2_tps_act_file, sep='\t', header=None, names=['ORF','Symbol'])
rck2_tps_act_targs = set(rck2_tps_act_df['ORF'].values)
assert len(rck2_tps_act_targs) == 4

rck2_tps_act_targs_def = rck2_def_prots.intersection(rck2_tps_act_targs)
print('\nrck2 defect Kanshin and TPS Rck2 activated target: {}'.format(len(rck2_tps_act_targs_def)))
print('{} of rck2 defect'.format(float(len(rck2_tps_act_targs_def))/len(rck2_def_prots)))
print('{} of TPS Rck2 directed target'.format(float(len(rck2_tps_act_targs_def))/len(rck2_tps_act_targs)))
for orf in sorted(rck2_tps_act_targs_def):
    print('{}\t{}').format(orf, rev_id_map[orf])

rck2_tps_act_targs_def_kanshin = kanshin_rck2_def_prots.intersection(rck2_tps_act_targs)
print('\nrck2 defect Kanshin and TPS Rck2 activated target: {}'.format(len(rck2_tps_act_targs_def_kanshin)))
print('{} of rck2 defect'.format(float(len(rck2_tps_act_targs_def_kanshin))/len(kanshin_rck2_def_prots)))
print('{} of TPS Rck2 directed target'.format(float(len(rck2_tps_act_targs_def_kanshin))/len(rck2_tps_act_targs)))
for orf in sorted(rck2_tps_act_targs_def_kanshin):
    print('{}\t{}').format(orf, rev_id_map[orf])

rck2 defect and TPS Rck2 directed target: 7
0.0343137254902 of rck2 defect
0.538461538462 of TPS Rck2 directed target
YBL007C	SLA1
YHR131C	YHR131C
YLR257W	YLR257W
YNL074C	MLF3
YNL267W	PIK1
YNR047W	FPK1
YOR018W	ROD1

rck2 defect Kanshin and TPS Rck2 directed target: 7
0.0362694300518 of rck2 defect
0.538461538462 of TPS Rck2 directed target
YBL007C	SLA1
YHR131C	YHR131C
YLR257W	YLR257W
YNL074C	MLF3
YNL267W	PIK1
YNR047W	FPK1
YOR018W	ROD1

rck2 defect Kanshin and TPS Rck2 activated target: 4
0.0196078431373 of rck2 defect
1.0 of TPS Rck2 directed target
YLR257W	YLR257W
YNL267W	PIK1
YNR047W	FPK1
YOR018W	ROD1

rck2 defect Kanshin and TPS Rck2 activated target: 4
0.020725388601 of rck2 defect
1.0 of TPS Rck2 directed target
YLR257W	YLR257W
YNL267W	PIK1
YNR047W	FPK1
YOR018W	ROD1


## Log the Python environment

In [12]:
! conda list

# packages in environment at C:\Program Files\Anaconda:
#
_license                  1.1                      py27_0    <unknown>
anaconda                  2.1.0                np19py27_0    <unknown>
argcomplete               0.8.1                    py27_0    <unknown>
asn1crypto                0.22.0                   py27_0  
astropy                   0.4.2                np19py27_0    <unknown>
atom                      0.3.9                    py27_0    <unknown>
beautiful-soup            4.3.2                    py27_0    <unknown>
binstar                   0.7.1                    py27_0    <unknown>
bitarray                  0.8.1                    py27_1    <unknown>
blaze                     0.6.3                np19py27_0    <unknown>
blz                       0.6.2                np19py27_0    <unknown>
bokeh                     0.6.1                np19py27_0    <unknown>
boto                      2.32.1                   py27_0    <unknown>
brewer2mpl                1.4.